In [49]:
import pandas as pd
import struct
import datetime
import pytz 
from datetime import datetime
import os

In [50]:
def to_str(x, lens):
    x_str = "{:.2f}".format(x)
    if (x // 100) == 0:
        # lon=format(lon, '.2f')
        return x_str.zfill(lens)

    elif ((-x) // 100) == 0:
        # lon = format(lon, '.2f')
        return x_str.zfill(lens + 1)
    else:
        return x_str

In [51]:
df_station=pd.read_csv("station.csv")
df_station['Region']=df_station['Region'].fillna('')
df_station['Country2']=df_station['Country2'].fillna('')
df_station['ElevationMeters']=df_station['ElevationMeters'].astype(int)
df_station['Region']=df_station['Region'].astype(str)
df_station['Country2']=df_station['Country2'].astype(str)
df_station.reset_index(inplace=True)
df_station['WhoAmI'] =   '+' + df_station['Latitude'].apply(to_str, args=(5,))+df_station['Longitude'].apply(to_str, args=(6,))+'/'
df_station['WhoAmI'].drop_duplicates(inplace=True)
df_station = df_station.sort_values(by=['WhoAmI'])
LOC=df_station['WhoAmI'].nunique()
df_station

,index,WhoAmI,Longitude,Latitude,ElevationMeters,ICAO,WMO,Region,Country2
38496,38496,+24.00-060.00/,-60.00,24.0,0,NaN,0,,
38359,38359,+24.00-060.25/,-60.25,24.0,0,NaN,0,,
38222,38222,+24.00-060.50/,-60.50,24.0,0,NaN,0,,
38085,38085,+24.00-060.75/,-60.75,24.0,0,NaN,0,,
37948,37948,+24.00-061.00/,-61.00,24.0,0,NaN,0,,
...,...,...,...,...,...,...,...,...,...
548,548,+58.00-129.00/,-129.00,58.0,1325,NaN,0,BC,CA
411,411,+58.00-129.25/,-129.25,58.0,1318,NaN,0,BC,CA
274,274,+58.00-129.50/,-129.50,58.0,1336,NaN,0,BC,CA
137,137,+58.00-129.75/,-129.75,58.0,1252,NaN,0,BC,CA


In [64]:
# Define the conversion function # Excel's epoch starts on "January 0, 1900"
def datetime_to_excel_double(date_obj):
    excel_epoch = datetime(1899, 12, 31, tzinfo=pytz.UTC)  # Make it timezone-aware by setting the timezone to UTC
    delta = date_obj - excel_epoch
    excel_date = delta.days + (delta.seconds / 86400.0) + (date_obj.microsecond / 86400e6)+1
    return excel_date


file_path = "20240117.csv" ###############################################This is the only place needs to be changed manually for all
df=pd.read_csv(file_path) 
# Fill NaN values with the previous non-null value
#df['WhoAmI'] =   '+' + df['lat'].apply(to_str, args=(5,))+df['long'].apply(to_str, args=(6,))+'/'
df

C:\Users\fsafdarian\AppData\Local\Temp\ipykernel_18096\3262807590.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


,Unnamed: 0,UTCISO8601,WhoAmI,DewPointF,tempF,GlobalHorizontalIrradianceWM2,CloudCoverPerc,DirectHorizontalIrradianceWM2,WindSpeedmph,WindDirection,WindSpeed100mph
0,0,2024-01-17T00:00:00.000Z,+58.00-130.00/,1,5,51,43,27,2,0,3
1,2363136,NaN,+34.75-085.25/,7,19,0,1,0,10,332,15
2,3614400,NaN,+30.00-061.50/,68,72,0,54,0,14,167,16
3,1305312,NaN,+49.50-105.25/,-11,0,7,22,3,13,300,23
4,2363232,NaN,+34.50-085.25/,7,21,0,1,0,11,334,16
...,...,...,...,...,...,...,...,...,...,...,...
3695707,3254399,NaN,+43.00-068.25/,19,27,0,100,0,16,323,16
3695708,432287,NaN,+28.50-122.00/,62,64,489,16,393,13,259,15
3695709,1096991,NaN,+44.25-109.25/,24,33,157,98,40,2,287,5
3695710,2576063,NaN,+28.50-081.25/,30,53,50,0,31,4,5,10


In [65]:

df['UTCISO8601'] = df['UTCISO8601'].fillna(method='ffill')

df = df.fillna(method='ffill') #just for 197605
# Convert UTCISO8601 to timestamp
df['UTCISO8601'] = pd.to_datetime(df['UTCISO8601'])
df['excel_double_datetime'] = df['UTCISO8601'].apply(datetime_to_excel_double)
df = df.sort_values(by=['excel_double_datetime', 'WhoAmI'])
COUNT=df['excel_double_datetime'].nunique()
unique_dates = df['UTCISO8601'].unique()
df['tempF102'] = df['tempF'] + 115
df['DewPointF104'] = df['DewPointF'] + 115
df['WindDirection107'] = df['WindDirection'] /5
df['GlobalHorizontalIrradianceWM2_120'] = df['GlobalHorizontalIrradianceWM2'] /5
df['DirectHorizontalIrradianceWM2_121'] = df['DirectHorizontalIrradianceWM2'] /5
df['WindDirection107'] = df['WindDirection107'].astype(int)
df['GlobalHorizontalIrradianceWM2_120'] = df['GlobalHorizontalIrradianceWM2_120'].astype(int)
df['DirectHorizontalIrradianceWM2_121'] = df['DirectHorizontalIrradianceWM2_121'].astype(int)
df['tempF102']=df['tempF102'].astype(int) #just for 197605
df['WindSpeedmph']=df['WindSpeedmph'].astype(int) #just for 197605
df['CloudCoverPerc']=df['CloudCoverPerc'].astype(int) #just for 197605
df['WindSpeed100mph']=df['WindSpeed100mph'].astype(int) #just for 197605
#df['GlobalHorizontalIrradianceWM2_120']=df['GlobalHorizontalIrradianceWM2_120'].fillna(method='ffill')
df

,Unnamed: 0,UTCISO8601,WhoAmI,DewPointF,tempF,GlobalHorizontalIrradianceWM2,CloudCoverPerc,DirectHorizontalIrradianceWM2,WindSpeedmph,WindDirection,WindSpeed100mph,excel_double_datetime,tempF102,DewPointF104,WindDirection107,GlobalHorizontalIrradianceWM2_120,DirectHorizontalIrradianceWM2_121
33738,3695616,2024-01-17 00:00:00+00:00,+24.00-060.00/,66,76,0,50,0,17,108,18,45308.000000,191,181,21,0,0
27611,3682464,2024-01-17 00:00:00+00:00,+24.00-060.25/,66,77,0,60,0,16,109,18,45308.000000,192,181,21,0,0
5795,3669312,2024-01-17 00:00:00+00:00,+24.00-060.50/,66,77,0,67,0,16,110,18,45308.000000,192,181,22,0,0
3033,3656160,2024-01-17 00:00:00+00:00,+24.00-060.75/,66,77,0,69,0,16,112,18,45308.000000,192,181,22,0,0
11660,3643008,2024-01-17 00:00:00+00:00,+24.00-061.00/,66,77,0,67,0,16,113,18,45308.000000,192,181,22,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3683123,52703,2024-01-20 23:00:00+00:00,+58.00-129.00/,-12,-5,72,99,22,2,357,2,45311.958333,110,103,71,14,4
3675071,39551,2024-01-20 23:00:00+00:00,+58.00-129.25/,-11,-5,73,98,23,2,27,4,45311.958333,110,104,5,14,4
3685454,26399,2024-01-20 23:00:00+00:00,+58.00-129.50/,-10,-4,77,97,24,3,49,5,45311.958333,111,105,9,15,4
3670405,13247,2024-01-20 23:00:00+00:00,+58.00-129.75/,-9,-3,75,98,23,3,65,4,45311.958333,112,106,13,15,4


In [66]:
# Extract filename without extension
file_name = os.path.basename(file_path)  
file_name_without_extension = os.path.splitext(file_name)[0]  
file_name_without_extension

'20240117'

In [67]:
def to_ascii_null_terminated(s):
    try:
        # Encode to ASCII and add a null terminator
        return s.encode('ascii') + b'\x00'
    except UnicodeEncodeError:
        # Handle the case where the string cannot be encoded in ASCII
        # This will replace non-ASCII characters with '?', and then add a null terminator
        return s.encode('ascii', 'replace') + b'\x00'

    
    
df_station['ascii_null_terminated_WhoAmI'] = df_station['WhoAmI'].apply(to_ascii_null_terminated)   
df_station['ascii_null_terminated_Region'] = df_station['Region'].apply(to_ascii_null_terminated) 
df_station['ascii_null_terminated_Country2'] = df_station['Country2'].apply(to_ascii_null_terminated) 
df_station


,index,WhoAmI,Longitude,Latitude,ElevationMeters,ICAO,WMO,Region,Country2,ascii_null_terminated_WhoAmI,ascii_null_terminated_Region,ascii_null_terminated_Country2
38496,38496,+24.00-060.00/,-60.00,24.0,0,NaN,0,,,b'+24.00-060.00/\x00',b'\x00',b'\x00'
38359,38359,+24.00-060.25/,-60.25,24.0,0,NaN,0,,,b'+24.00-060.25/\x00',b'\x00',b'\x00'
38222,38222,+24.00-060.50/,-60.50,24.0,0,NaN,0,,,b'+24.00-060.50/\x00',b'\x00',b'\x00'
38085,38085,+24.00-060.75/,-60.75,24.0,0,NaN,0,,,b'+24.00-060.75/\x00',b'\x00',b'\x00'
37948,37948,+24.00-061.00/,-61.00,24.0,0,NaN,0,,,b'+24.00-061.00/\x00',b'\x00',b'\x00'
...,...,...,...,...,...,...,...,...,...,...,...,...
548,548,+58.00-129.00/,-129.00,58.0,1325,NaN,0,BC,CA,b'+58.00-129.00/\x00',b'BC\x00',b'CA\x00'
411,411,+58.00-129.25/,-129.25,58.0,1318,NaN,0,BC,CA,b'+58.00-129.25/\x00',b'BC\x00',b'CA\x00'
274,274,+58.00-129.50/,-129.50,58.0,1336,NaN,0,BC,CA,b'+58.00-129.50/\x00',b'BC\x00',b'CA\x00'
137,137,+58.00-129.75/,-129.75,58.0,1252,NaN,0,BC,CA,b'+58.00-129.75/\x00',b'BC\x00',b'CA\x00'


In [68]:
aPWWVersion = 1
aPWWFileName = file_name_without_extension + ".pww"  

# Extracting the smallest start time and the largest end time from the DataFrame and converting them to timestamps
aStartDateTimeUTC = df['excel_double_datetime'].min()
aEndDateTimeUTC = df['excel_double_datetime'].max()
#area=[58, -130, 24, -60] North 58°, West -130°, South 24°, East -60°  
aMinLat=int(df_station['Latitude'].min()) 
aMaxLat= int(df_station['Latitude'].max()) 
aMinLon=int(df_station['Longitude'].min())
aMaxLon=int(df_station['Longitude'].max())
# Define the optional identifier field count
LOC_FC = 0  # for extra loc variables from table 1
VARCOUNT = 8  # Set this to the number of weather variable types you have

    
    
with open(aPWWFileName, 'wb') as file:
    file.write(struct.pack('<h', 2001))
    file.write(struct.pack('<h', 8065))
    file.write(struct.pack('<h', aPWWVersion))
    file.write(struct.pack('<d', aStartDateTimeUTC))
    file.write(struct.pack('<d', aEndDateTimeUTC))
    file.write(struct.pack('<d', aMinLat))
    file.write(struct.pack('<d', aMaxLat))
    file.write(struct.pack('<d', aMinLon))
    file.write(struct.pack('<d', aMaxLon))
    file.write(struct.pack('<h', 0))
    file.write(struct.pack('<i', COUNT))    #countNumber of datetime values (COUNT)
    file.write(struct.pack('<i', 3600))
    file.write(struct.pack('<i', LOC)) #Number of weather measurement locations (LOC)
    file.write(struct.pack('<h', LOC_FC)) #Loc_FC # Pack the data into INT16 format and write to stream
    file.write(struct.pack('<h', VARCOUNT))


    
    # Temp in F
    file.write(struct.pack('<h', 102))
    # Dew point in F
    file.write(struct.pack('<h', 104))
    # Wind speed at surface (10m) in mph
    file.write(struct.pack('<h', 106))
    # Wind direction at surface (10m) in 5-degree increments
    file.write(struct.pack('<h', 107))
    # Total cloud cover percentage
    file.write(struct.pack('<h', 119))
    # Wind speed at 100m in mph
    file.write(struct.pack('<h', 110))
    # Global Horizontal Irradiance in W/m^2 divided by 4
    file.write(struct.pack('<h', 120))
    # Direct Horizontal Irradiance in W/m^2 divided by 4
    file.write(struct.pack('<h', 121))
    file.write(struct.pack('<h', 8)) #BYTECOUNT
    for row in df_station.index:
        # Write Latitude (DOUBLE)
        file.write(struct.pack('<d', df_station['Latitude'][row]))
        # Write Longitude (DOUBLE)
        file.write(struct.pack('<d', df_station['Longitude'][row])) 
        # Write AltitudeM (INT16)
        file.write(struct.pack('<h', df_station['ElevationMeters'][row]))
        # Write Name (STRING)
        file.write(df_station['ascii_null_terminated_WhoAmI'][row])
        file.write(df_station['ascii_null_terminated_Country2'][row])
        file.write(df_station['ascii_null_terminated_Region'][row])

    for date in unique_dates:
        # Filter rows by unique date
        rows = df[df['UTCISO8601'] == date]
        
        for temp in rows['tempF102']:
            #file.write(temp.encode('utf-8'))
            #file.write(struct.pack('<i', temp))
            file.write(temp.to_bytes(1, 'little'))
        
        for dew_point in rows['DewPointF104']:
            #file.write(struct.pack('<b', dew_point.to_bytes(1, 'little')))
            file.write(dew_point.to_bytes(1, 'little'))
        for wind_speed in rows['WindSpeedmph']:
            #file.write(struct.pack('<b', wind_speed.to_bytes(1, 'little')))
            file.write(wind_speed.to_bytes(1, 'little'))
       
        for wind_direction in rows['WindDirection107']:
            #file.write(struct.pack('d', wind_direction))
            file.write(wind_direction.to_bytes(1, 'little'))
                  
        for CloudCoverPerc in rows['CloudCoverPerc']:
            #file.write(struct.pack('<b', CloudCoverPerc.to_bytes(1, 'little')))
            file.write(CloudCoverPerc.to_bytes(1, 'little'))
        
        for WindSpeed100mph in rows['WindSpeed100mph']:
            #file.write(struct.pack('<b', WindSpeed100mph.to_bytes(1, 'little')))
            file.write(WindSpeed100mph.to_bytes(1, 'little'))
           
        for GlobalHorizontalIrradianceWM2_120 in rows['GlobalHorizontalIrradianceWM2_120']:
            #file.write(struct.pack('d', GlobalHorizontalIrradianceWM2_120))
            file.write(GlobalHorizontalIrradianceWM2_120.to_bytes(1, 'little'))
        
        for DirectHorizontalIrradianceWM2_121 in rows['DirectHorizontalIrradianceWM2_121']:
            #file.write(struct.pack('d', DirectHorizontalIrradianceWM2_121))
            file.write(DirectHorizontalIrradianceWM2_121.to_bytes(1, 'little'))

   
    